In [1]:
from __future__ import print_function

import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
#5375-40

seed =776
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = 11
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)



class Dataset(data.Dataset):
    def __init__(self, X1,Y):
        self.X1 = X1
        self.Y = Y
    def __len__(self):        
        return len(self.X1)
    
    def __getitem__(self, index):
        x = self.X1[index]
        y = self.Y[index]
        return x,y



## Negetive pictures

In [2]:
xTrain=np.load("../extracted_features/FS/input_features_free_train.npy")

yTrain=np.load("../extracted_features/FS/label_free_train.npy")

xTest=np.load("../extracted_features/FS/input_features_free_test.npy")

yTest=np.load("../extracted_features/FS/label_free_test.npy")

xVal=np.load("../extracted_features/FS/input_features_free_val.npy")

yVal=np.load("../extracted_features/FS/label_free_val.npy")


In [3]:
xTrain.shape

(1121, 19, 24)

In [4]:
xVal.shape

(199, 19, 24)

In [5]:
xTest.shape

(190, 19, 24)

In [6]:
xTrain = xTrain.reshape(len(xTrain),1,19,24)
xTest = xTest.reshape(len(xTest),1,19,24)
xVal = xVal.reshape(len(xVal),1,19,24)

In [7]:
print(yTest)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [8]:
print(xTrain.shape,yTrain.shape)
print(xTest.shape,yTest.shape)
print(xVal.shape,yVal.shape)

(1121, 1, 19, 24) (1121,)
(190, 1, 19, 24) (190,)
(199, 1, 19, 24) (199,)


In [9]:
traindataset = Dataset(xTrain,yTrain)
testdataset = Dataset(xTest,yTest)
valdataset = Dataset(xVal,yVal)

In [10]:
class Stammering(data_utils.Dataset):
    def __init__(self, target_number=1, mean_bag_length=5, var_bag_length=2, num_bag=150, seed=2021, train="train"):
        self.target_number = target_number
        self.mean_bag_length = mean_bag_length
        self.var_bag_length = var_bag_length
        self.num_bag = num_bag
        self.train = train
        self.r = np.random.RandomState(seed)

        if self.train=="train":
            self.train_bags_list, self.train_labels_list = self._create_bags()
        elif self.train=="val":
            self.val_bags_list, self.val_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _create_bags(self):
        if self.train=="train":
            print("train")
            loader = data_utils.DataLoader(traindataset,
                                           batch_size=1,
                                           shuffle=True)
        elif self.train=="val":
            print("val")
            loader = data_utils.DataLoader(valdataset,
                                           batch_size=1,
                                           shuffle=True)
        else:
            loader = data_utils.DataLoader(testdataset,
                                           batch_size=1,
                                           shuffle=True)
            
        bags_list = []
        labels_list = []
        for (batch_data, batch_labels) in loader:

            bags_list.append(batch_data.reshape(19,1,24))
            temp = torch.as_tensor(np.array([batch_labels for x in range(19)]))
            labels_list.append(temp)
            
               

        return bags_list, labels_list

    def __len__(self):
        if self.train=="train":
            return len(self.train_labels_list)
        elif self.train=="val":
            return len(self.val_labels_list)
        else:
            return len(self.test_labels_list)

    def __getitem__(self, index):
        if self.train=="train":
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]), self.train_labels_list[index]]
        elif self.train=="val":
            bag = self.val_bags_list[index]
            label = [max(self.val_labels_list[index]), self.val_labels_list[index]]
        else:
            bag = self.test_bags_list[index]
            label = [max(self.test_labels_list[index]), self.test_labels_list[index]]

        return bag, label

In [11]:
train_loader = data_utils.DataLoader(Stammering(train="train"),num_workers=0,worker_init_fn=seed_worker,batch_size=1,shuffle=True)
test_loader = data_utils.DataLoader(Stammering(train="val"),num_workers=0,worker_init_fn=seed_worker,batch_size=1,shuffle=True)
val_loader = data_utils.DataLoader(Stammering(train="test"),num_workers=0,worker_init_fn=seed_worker,batch_size=1,shuffle=True)

train
val


In [12]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if type(m) == nn.Conv1d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
        
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 128
        self.D = 64
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Conv1d(1, 256, kernel_size=2),
            nn.ReLU(),
            nn.Conv1d(256, 512, kernel_size=2),
            nn.ReLU(),
            nn.Conv1d(512, 128, kernel_size=2),
            nn.ReLU(),
            nn.Conv1d(128, 32, kernel_size=2),
            nn.ReLU()
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.BatchNorm1d(640, affine=False),
            nn.Linear(640, 512),
            nn.ReLU(),
            nn.Linear(512,self.L),
            nn.ReLU()
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(

            nn.Linear(self.L*self.K, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        x=x.squeeze(0)

        H = self.feature_extractor_part1(x)
        
        H = H.view(H.size(0), -1)

        H = self.feature_extractor_part2(H)  

        A = self.attention(H)

        A = torch.transpose(A, 1, 0) 

        A = F.softmax(A, dim=1)  

        M = torch.mm(A, H)  

        Y_prob = self.classifier(M)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A




    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A

In [13]:
# from torchsummary import summary
# model=Attention()
# summary=summary(model.cuda(),(1,24))

In [14]:
LR = 1e-4
model=Attention()
if torch.cuda.is_available():model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

best_model = "./saved_models/best_model_Attention-mil-fs-grp3"
train_acc = []
modelname=[]
val_acc = []
best_acc = 0
for epoch in range(0, 50): #15,#15,200
    model.train()
    train_loss = 0.
    train_error = 0.
    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(train_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data.float(), bag_label.float())
        train_loss += loss.data[0]
        error,y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        train_error += error
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
        loss.mean().backward()
        optimizer.step()

    
    trainacc = accuracy_score(y,ypred)
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print("EPOCH ",epoch)
    print('Train : Loss: {:.4f}, Train error: {:.4f}, Train f1 : {}'.format(train_loss.cpu().numpy()[0], 
                                                                                train_error,trainacc))
    y =[]
    ypred = []
    val_error=0.
    val_loss= 0.
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(val_loader):
            bag_label = label[0]
            data, bag_label = data.cuda(), bag_label.cuda()
            data, bag_label = Variable(data), Variable(bag_label)
            loss, _ = model.calculate_objective(data.float(), bag_label.float())
            val_loss += loss.data[0]
            error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
            val_error += error
            y_pred=y_pred.squeeze(1)
            ypred.extend(y_pred.tolist())
            y.extend(bag_label.tolist())
        valacc = accuracy_score(y,ypred)
        val_loss /= len(val_loader)
        val_error /= len(val_loader)
        print('Val : Loss: {:.4f}, val error: {:.4f}, Val f1 :{}'.format(val_loss.cpu().numpy()[0], val_error,valacc))
        if valacc>=best_acc:
            print("---------State saved---------")
            best_acc = valacc
            best_state=model.state_dict()
            torch.save(best_state, best_model+'_epoch_'+str(epoch)+".pth")
            modelname.append(best_model+'_epoch_'+str(epoch)+".pth")
        print('Best validation accuracy ',best_acc)

        

EPOCH  0
Train : Loss: 0.6887, Train error: 0.4817, Train f1 : 0.5182872435325602
Val : Loss: 2.1729, val error: 0.4579, Val f1 :0.5421052631578948
---------State saved---------
Best validation accuracy  0.5421052631578948
EPOCH  1
Train : Loss: 0.6497, Train error: 0.4326, Train f1 : 0.567350579839429
Val : Loss: 1.9664, val error: 0.4158, Val f1 :0.5842105263157895
---------State saved---------
Best validation accuracy  0.5842105263157895
EPOCH  2
Train : Loss: 0.6318, Train error: 0.4273, Train f1 : 0.5727029438001784
Val : Loss: 3.3035, val error: 0.5368, Val f1 :0.4631578947368421
Best validation accuracy  0.5842105263157895
EPOCH  3
Train : Loss: 0.6256, Train error: 0.4005, Train f1 : 0.5994647636039251
Val : Loss: 2.0707, val error: 0.4947, Val f1 :0.5052631578947369
Best validation accuracy  0.5842105263157895
EPOCH  4
Train : Loss: 0.6181, Train error: 0.3996, Train f1 : 0.6003568242640499
Val : Loss: 3.0714, val error: 0.4421, Val f1 :0.5578947368421052
Best validation accur

EPOCH  42
Train : Loss: 0.3123, Train error: 0.1347, Train f1 : 0.8652988403211418
Val : Loss: 5.4529, val error: 0.4737, Val f1 :0.5263157894736842
Best validation accuracy  0.6157894736842106
EPOCH  43
Train : Loss: 0.2847, Train error: 0.1249, Train f1 : 0.8751115075825157
Val : Loss: 5.7929, val error: 0.5421, Val f1 :0.45789473684210524
Best validation accuracy  0.6157894736842106
EPOCH  44
Train : Loss: 0.2756, Train error: 0.1151, Train f1 : 0.8849241748438894
Val : Loss: 4.5585, val error: 0.4211, Val f1 :0.5789473684210527
Best validation accuracy  0.6157894736842106
EPOCH  45
Train : Loss: 0.2752, Train error: 0.1195, Train f1 : 0.8804638715432649
Val : Loss: 3.6307, val error: 0.4368, Val f1 :0.5631578947368421
Best validation accuracy  0.6157894736842106
EPOCH  46
Train : Loss: 0.2661, Train error: 0.1213, Train f1 : 0.8786797502230151
Val : Loss: 5.2710, val error: 0.5368, Val f1 :0.4631578947368421
Best validation accuracy  0.6157894736842106
EPOCH  47
Train : Loss: 0.247

In [15]:
device = torch.device("cuda")
model=Attention()
best_state=torch.load(modelname[-1]) #../best-models/Attention_MIL_best_free_speech_raw_F1_79.pth
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.

with torch.no_grad():
    
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        instance_labels = label[1]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, attention_weights = model.calculate_objective(data.float(), bag_label.float())
        test_loss += loss.data[0]
        error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
        test_error += error
        predicted_label=predicted_label.squeeze(1)
        ypred.extend(predicted_label.tolist())
        y.extend(bag_label.tolist())

    acc=accuracy_score(y,ypred)
    tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
    f1score=f1_score(y,ypred)
    precision=precision_score(y,ypred)
    recall=recall_score(y,ypred)
    roc=roc_auc_score(y,ypred)
    specificity=tn/(tn+fp)
    print(acc,f1score,precision,recall,roc,specificity)
    

0.7236180904522613 0.7263681592039801 0.6576576576576577 0.8111111111111111 0.731243628950051 0.6513761467889908


In [16]:
for i in range(len(modelname)):
    device = torch.device("cuda")
    model=Attention()
    best_state=torch.load(modelname[i]) #../best-models/Attention_MIL_best_free_speech_raw_F1_79.pth
    model.load_state_dict(best_state)
    model.to(device)
    model.eval()
    test_loss = 0.
    correct = 0
    total = 0
    y =[]
    ypred = []
    test_error = 0.

    with torch.no_grad():

        for batch_idx, (data, label) in enumerate(test_loader):
            bag_label = label[0]
            instance_labels = label[1]
            data, bag_label = data.cuda(), bag_label.cuda()
            data, bag_label = Variable(data), Variable(bag_label)
            loss, attention_weights = model.calculate_objective(data.float(), bag_label.float())
            test_loss += loss.data[0]
            error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
            test_error += error
            predicted_label=predicted_label.squeeze(1)
            ypred.extend(predicted_label.tolist())
            y.extend(bag_label.tolist())

        acc=accuracy_score(y,ypred)
        tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
        f1score=f1_score(y,ypred)
        precision=precision_score(y,ypred)
        recall=recall_score(y,ypred)
        roc=roc_auc_score(y,ypred)
        specificity=tn/(tn+fp)
        print(acc,f1score,precision,recall,roc,specificity)  
    

0.678391959798995 0.728813559322034 0.589041095890411 0.9555555555555556 0.7025484199796127 0.44954128440366975
0.48743718592964824 0.2388059701492537 0.36363636363636365 0.17777777777777778 0.46044852191641183 0.7431192660550459
0.5879396984924623 0.6203703703703705 0.5317460317460317 0.7444444444444445 0.6015800203873598 0.45871559633027525
0.49748743718592964 0.3055555555555555 0.4074074074074074 0.24444444444444444 0.47543323139653415 0.7064220183486238
0.7236180904522613 0.7263681592039801 0.6576576576576577 0.8111111111111111 0.731243628950051 0.6513761467889908


In [17]:
modelname[-1]
#9639-.628140703517588 0.5795454545454546 0.5930232558139535 0.5666666666666667 0.622782874617737 0.6788990825688074

'./saved_models/best_model_Attention-mil-fs-grp3_epoch_32.pth'